In [ ]:


# DIMS = (1, 28, 28)
# OUTPUT_DIMS = (1,)
# MAPPING = list(range(10))
# TRAIN_SIZE = 55000
# VAL_SIZE = 5000

class MB2016(BaseDataModule):
    """MB2016 DataModule."""

    def __init__(self, args=None) -> None:
        super().__init__(args)

        # self.data_dir = metadata.DOWNLOADED_DATA_DIRNAME
        # self.transform = MNISTStem()
        # self.input_dims = metadata.DIMS
        # self.output_dims = OUTPUT_DIMS
        # self.mapping = metadata.MAPPING

    def prepare_data(self) -> None:

        # check if data has already been prepared
        if self.data_filename.exists():
            return

        #extract dataset to pandas dataframe
        with temporary_working_directory(RAW_DATA_DIRNAME):
            with open(self.data_filename, 'rb') as pkl_file:
                pickle_data = pickle.load(pkl_file)
                data = pd.DataFrame.from_dict(pickle_data).T

        #remove climbs with low or no amounts of repeats
        cleaned_data = data[data['repeats'] >= MIN_REPEATS]

        #remove climbs with grades that contain less than a minimum number of climbs
        grade_count = cleaned_data['grade'].value_counts()
        grades = [key for key, value in grade_count.items() if value>= MIN_GRADE_COUNT]
        cleaned_data = cleaned_data[cleaned_data['grade'].isin(grades)]

        # max length >>> TODO Clean remove climbs with crazy number of holds
        # add <S> <SM> <ME> <P> <P> <P> for x input
        # mapping for [1,2] etc required > transform to tensor wigth mapped values
        # for

    def setup(self) -> None:
        # Cross validation TODO

        # Grade stratified test and train split
        split = StratifiedShuffleSplit(n_splits=1, test_size = 0.2, random_state=42)
        for train_index, test_index in split.split(cleaned_data,cleaned_data['grade']):
          strat_train_set = cleaned_data.iloc[train_index]
          strat_test_set = cleaned_data.iloc[test_index]

        """
        pipeline for X input
        - weights
        - list of lists > list of tokens (use tuples?)
        - token has an embedding mapped to it


        """


        # Token dictinary
        # drop repeats with only 1 repeat, and v12 +


        X = data.loc[:,['start', 'mid', 'end']]


        X['start'] = X['start'].map(lambda x: [ str(x1) for x1 in x])

        # X['start'] = X['start'].map(lambda x: np.asarray(x))
        # X['mid'] = X['mid'].map(lambda x: np.asarray(x))
        # X['end'] = X['end'].map(lambda x: np.asarray(x))

        display(X.start[0])
        display(X)

        X.info(verbose=True)


        X['mid'] = X['mid'].apply(lambda x: np.ravel(x))
        X['end'] = X['end'].apply(lambda x: np.ravel(x))

        Y = data.loc[:,'grade']
        lb = LabelBinarizer()
        Y = torch.Tensor(lb.fit_transform(Y))

    @property
    def data_filename(self):
        return (
            PROCESSED_DATA_DIRNAME /
            f"minr_{self.min_repeats}_mingc{self.min_grade_count:f}_maxs{self.max_start_holds:f}"
            f"minm_{self.min_mid_holds}_maxm{self.max_mid_holds:f}_maxe{self.max_end_holds:f}"
            f"_ntr{self.num_train}_ntv{self.num_val}_nte{self.num_test}_{self.with_start_mid_end_tokens}.csv"
        )


data_holds = MB2016()


In [ ]:
import argparse
import pickle
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelBinarizer
from torch.utils.data import random_split
import itertools
from itertools import permutations

from grade_predictor.data.base_data_module import BaseDataModule, load_and_print_info
from grade_predictor.util import temporary_working_directory

from sklearn.preprocessing import LabelBinarizer
import grade_predictor.metadata.transformer as metadata


""" Metada """

RAW_DATA_DIRNAME = metadata.RAW_DATA_DIRNAME
RAW_DATA_FILENAME = metadata.RAW_DATA_FILENAME
DATA_DIRNAME = metadata.DATA_DIRNAME
PROCESSED_DATA_DIRNAME = metadata.PROCESSED_DATA_DIRNAME

NUM_SPECIAL_TOKENS = 0

MIN_REPEATS = 2
MIN_GRADE_COUNT = 50
MAX_START_HOLDS = 2
MIN_MID_HOLDS = 2
MAX_MID_HOLDS = 11
MAX_END_HOLDS = 2
MAX_SEQUENCE = MAX_START_HOLDS + MAX_MID_HOLDS + MAX_END_HOLDS
NUM_TRAIN = 100
NUM_VAL = 20
NUM_TEST = 10


class MB2016(BaseDataModule):
    """MB2016 dataset: Scraped Moonboard 2016 climbs information"""

    def __init__(self, args=None) -> None:

        super().__init__(args)

        self.min_repeats = self.args.get("min_repeats", MIN_REPEATS)
        self.min_grade_count = self.args.get("min_grade_count", MIN_GRADE_COUNT)
        self.max_start_holds = self.args.get("max_start_holds", MAX_START_HOLDS)
        self.min_mid_holds = self.args.get("min_mid_holds", MIN_MID_HOLDS)
        self.max_mid_holds = self.args.get("max_mid_holds", MAX_MID_HOLDS)
        self.max_end_holds = self.args.get("max_end_holds", MAX_END_HOLDS)

        self.with_start_mid_end_tokens = self.args.get("with_start_mid_end_tokens", False)

        self.num_train = self.args.get("num_train", NUM_TRAIN)
        self.num_val = self.args.get("num_val", NUM_VAL)
        self.num_test = self.args.get("num_test", NUM_TEST)




        self.data_dir = self.args.get("data_dir", DATA_DIRNAME)
        self.processed_data_dir = self.args.get("data_dir", PROCESSED_DATA_DIRNAME)
        # self.transform = MNISTStem()
        # self.input_dims = ()
        # self.output_dims = OUTPUT_DIMS
        # self.mapping = metadata.MAPPING

    def prepare_data(self) -> None:

        # check if data has already been prepared
        if self.data_filename.exists():
            return

        #extract dataset to pandas dataframe
        with temporary_working_directory(self.data_dir):
            with open(self.data_filename, 'rb') as pkl_file:
                pickle_data = pickle.load(pkl_file)
                data = pd.DataFrame.from_dict(pickle_data).T

        #Clean dataset of extraneous climbs
        cleaned_data = data[data['repeats'] >= MIN_REPEATS]
        cleaned_data = cleaned_data[cleaned_data.mid.map(len) <= MAX_MID_HOLDS]
        cleaned_data = cleaned_data[cleaned_data.mid.map(len) >= MIN_MID_HOLDS]
        cleaned_data = cleaned_data[cleaned_data.start.map(len) <= MAX_START_HOLDS]
        cleaned_data = cleaned_data[cleaned_data.end.map(len) <= MAX_END_HOLDS]

        grade_count = cleaned_data['grade'].value_counts()
        grades = [key for key, value in grade_count.items() if value>= MIN_GRADE_COUNT]
        cleaned_data = cleaned_data[cleaned_data['grade'].isin(grades)]
        #Save cleaned data
        with temporary_working_directory(self.processed_data_dir):
            cleaned_data.to_pickle(self.data_filename)



        # Create token dictionary for MoonBoard hold positions
        rows = range(0,11)
        columns = range(0,18)
        positions = itertools.product(rows, columns)
        token_dict = {}
        if self.with_start_mid_end_tokens:
            # TODO: add add <S> <SM> <ME> <P>
            pass
        for i,k in enumerate(positions, start=1):
            token_dict[k] = i

        self.X = _dataframe_to_token_array(cleaned_data.loc[:,['start', 'mid', 'end']])
        lb = LabelBinarizer()
        self.Y = data.loc[:,'grade']
        self.Y = torch.FloatTensor(lb.fit_transform(self.Y))






    def setup(self) -> None:
        # Cross validation TODO

        # Grade stratified test and train split
        split = StratifiedShuffleSplit(n_splits=1, test_size = 0.2, random_state=42)
        for train_index, test_index in split.split(cleaned_data,cleaned_data['grade']):
            strat_train_set = cleaned_data.iloc[train_index]
            strat_test_set = cleaned_data.iloc[test_index]

    @property
    def data_filename(self):
        return (
            PROCESSED_DATA_DIRNAME /
            f"minr_{self.min_repeats}_mingc{self.min_grade_count:f}_maxs{self.max_start_holds:f}"
            f"minm_{self.min_mid_holds}_maxm{self.max_mid_holds:f}_maxe{self.max_end_holds:f}"
            f"_ntr{self.num_train}_ntv{self.num_val}_nte{self.num_test}_{self.with_start_mid_end_tokens}.pkl"
        )



def _dataframe_to_token_array(df, max_sequence, token_dict):
    # token_array = torch.zeros(df.shape[0], 4, MAX_SEQUENCE)
    def _row_to_token_matrix(row):
        token_matrix = torch.zeros((4, max_sequence), dtype=torch.int)
        i = 0
        for position_column, position_index in [(row['start'], 1), (row['mid'], 2), (row['end'], 3)]:
            for item in position_column:
                token_matrix[0][i] = token_dict[tuple(item)]
                token_matrix[1][i] = position_index
                token_matrix[2][i] = tuple(item)[0]
                token_matrix[3][i] = tuple(item)[1]
                i +=1
        return token_matrix

    token_array = df.apply(lambda x: _row_to_token_matrix(x), axis=1)
    return token_array


# def _download_and_process_MB2016():
#     metadata = toml.load(METADATA_FILENAME)
#     _download_raw_dataset(metadata, DL_DATA_DIRNAME)
#     _process_raw_dataset(metadata["filename"], DL_DATA_DIRNAME)

def _sample_to_balance(x, y):
    """Because the dataset is not balanced, we take at most the mean number of instances per class."""
    np.random.seed(42)
    num_to_sample = int(np.bincount(y.flatten()).mean())
    all_sampled_inds = []
    for label in np.unique(y.flatten()):
        inds = np.where(y == label)[0]
        sampled_inds = np.unique(np.random.choice(inds, num_to_sample))
        all_sampled_inds.append(sampled_inds)
    ind = np.concatenate(all_sampled_inds)
    x_sampled = x[ind]
    y_sampled = y[ind]
    return x_sampled, y_sampled

load_and_print_info(MB2016)